[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/docs/quick-tour/interacting-with-the-index.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/docs/quick-tour/interacting-with-the-index.ipynb)

# Interacting with a Pinecone index

Pinecone creates an index for your input vectors,
and it lets you query their nearest neighbors.
A Pinecone index supports the following operations:

* `upsert`: insert data formatted as `(id, vector)` tuples into the index, or replace existing `(id, vector)` tuples with new vector values. Optionally, you can attach metadata for each vector so you can use them in the query by specifying conditions. The upserted vector will look like `(id, vector, metadata)`.
* `delete`: delete vectors by id.
* `query`: query the index and retrieve the top-k nearest neighbors based on dot-product, cosine-similarity, Euclidean distance, and more.
* `fetch`: fetch vectors stored in the index by id.
* `describe_index_stats`: get statistics about the index.

## Prerequisites

Install dependencies.

In [1]:
import os
import lantern_pinecone as pinecone

LANTERN_DB_URL = os.environ.get('LANTERN_DB_URL') or 'postgres://postgres@localhost:5432'
pinecone.init(LANTERN_DB_URL)

## Creating the index

In [2]:
index_name = "interacting-with-the-index"

In [3]:
import time

# Delete index if exists
if index_name in pinecone.list_indexes():
    pinecone.delete_index(index_name)

# Create index
pinecone.create_index(name=index_name, dimension=2, metric="euclidean")

# wait for index to be ready before connecting
while not pinecone.describe_index(index_name).status['ready']:
    time.sleep(1)

# Connect to the index
index = pinecone.Index(index_name=index_name)

### Insert vectors


In [4]:
# Generate some data
import pandas as pd

df = pd.DataFrame()
df["id"] = ["A", "B", "C", "D", "E"]
df["vector"] = [[1., 1.], [2., 2.], [3., 3.], [4., 4.], [5., 5.]]
df

,id,vector
0,A,"[1.0, 1.0]"
1,B,"[2.0, 2.0]"
2,C,"[3.0, 3.0]"
3,D,"[4.0, 4.0]"
4,E,"[5.0, 5.0]"


We perform upsert operations in our index. This call will insert a new vector in the index or update the vector if the id was already present.

In [5]:
# Upsert the vectors
AB_df = df[:2]
index.upsert(vectors=zip(AB_df.id, AB_df.vector))

2

### Fetch vectors

In [6]:
# Fetch vectors by ID
fetch_results = index.fetch(ids=["A", "B"])
fetch_results

{'namespace': '',
 'vectors': {'A': {'id': 'A', 'metadata': {}, 'values': [1.0, 1.0]},
  'B': {'id': 'B', 'metadata': {}, 'values': [2.0, 2.0]}}}

### Query top-k vectors

In [7]:
# Query top-k nearest neighbors
query_results = index.query([1.1, 1.1], top_k=2)
query_results

{'namespace': '',
 'matches': [{'id': 'A', 'score': 0.020000009, 'values': [], 'metadata': {}},
  {'id': 'B', 'score': 1.6199999, 'values': [], 'metadata': {}}]}

### Update vectors by ID

In [8]:
# Fetch current vectors by ID
fetch_result = index.fetch(ids=["A"])
fetch_result

{'namespace': '',
 'vectors': {'A': {'id': 'A', 'metadata': {}, 'values': [1.0, 1.0]}}}

In [9]:
# Update vectors by ID
index.upsert(vectors=[("A",[0., 0.])])

1

In [10]:
# Fetch vector by the same ID again
fetch_result = index.fetch(ids=["A"])
fetch_result

{'namespace': '',
 'vectors': {'A': {'id': 'A', 'metadata': {}, 'values': [1.0, 1.0]}}}

### Delete vectors by ID

In [11]:
# Delete vectors by ID
index.delete(ids=["A"])

In [12]:
# Deleted vectors are empty
fetch_results = index.fetch(ids=["A", "B"])
fetch_results

{'namespace': '',
 'vectors': {'B': {'id': 'B', 'metadata': {}, 'values': [2.0, 2.0]}}}

### Get index statistics

In [13]:
# Index statistics
index.describe_index_stats()

{'dimensions': 2,
 'index_fullness': 1.0,
 'total_count': 1,
 'namespaces': {'': {'vector_count': 1}}}

### Delete the index

In [14]:
# delete the index
pinecone.delete_index(index_name)